# <center>**`Project 1`**</center>

# <center>**`Weather Preduction by Machine Learning`**</center>

**In this Project, we will show how we can do Weather Forecasting with Machine Learning algorithm and compare some frameworks for further classification.**

In [1]:
import numpy as np # For Linear Algebra
import pandas as pd # To Work With Data

# for visualizations
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime # Time Series analysis.

**To look at first 5 rows of the data**

In [3]:
df = pd.read_csv("/content/Weather.csv")
df.head() # This will show us top 5 rows of the dataset by default.

,Unnamed: 0,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,0,1901,17.99,19.43,23.49,26.41,28.28,28.60,27.49,26.98,26.26,25.08,21.73,18.95
1,1,1902,19.00,20.39,24.10,26.54,28.68,28.44,27.29,27.05,25.95,24.37,21.33,18.78
2,2,1903,18.32,19.79,22.46,26.03,27.93,28.41,28.04,26.63,26.34,24.57,20.96,18.29
3,3,1904,17.77,19.39,22.95,26.73,27.83,27.85,26.84,26.73,25.84,24.36,21.07,18.84
4,4,1905,17.40,17.79,21.78,24.84,28.32,28.69,27.67,27.47,26.29,26.16,22.07,18.71


**We have got an unexpected column named Unnamed: 0. Well, this is a very common problem. We face this when our csv file has an index column which has no name. here is how we can get rid of it.**

In [4]:
df = pd.read_csv("Weather.csv", index_col=0)
df.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1901,17.99,19.43,23.49,26.41,28.28,28.60,27.49,26.98,26.26,25.08,21.73,18.95
1,1902,19.00,20.39,24.10,26.54,28.68,28.44,27.29,27.05,25.95,24.37,21.33,18.78
2,1903,18.32,19.79,22.46,26.03,27.93,28.41,28.04,26.63,26.34,24.57,20.96,18.29
3,1904,17.77,19.39,22.95,26.73,27.83,27.85,26.84,26.73,25.84,24.36,21.07,18.84
4,1905,17.40,17.79,21.78,24.84,28.32,28.69,27.67,27.47,26.29,26.16,22.07,18.71


**Now, we’ll make an attribute that would contain date (month, year). So that we could get temperature values with the timeline.**

In [5]:
df1 = pd.melt(df, id_vars='YEAR', value_vars=df.columns[1:])
df1.head()

,YEAR,variable,value
0,1901,JAN,17.99
1,1902,JAN,19.00
2,1903,JAN,18.32
3,1904,JAN,17.77
4,1905,JAN,17.40


In [6]:
df1['Date'] = df1['variable'] + ' ' + df1['YEAR'].astype(str)  
df1.loc[:,'Date'] = df1['Date'].apply(lambda x : datetime.strptime(x, '%b %Y')) ## Converting String to datetime object
df1.head()

,YEAR,variable,value,Date
0,1901,JAN,17.99,1901-01-01
1,1902,JAN,19.00,1902-01-01
2,1903,JAN,18.32,1903-01-01
3,1904,JAN,17.77,1904-01-01
4,1905,JAN,17.40,1905-01-01


# <center>**`Temperature through time`**</center>

In [7]:
df1.columns=['Year', 'Month', 'Temprature', 'Date']
df1.sort_values(by='Date', inplace=True) ## To get the time series right.
fig = go.Figure(layout = go.Layout(yaxis=dict(range=[0, df1['Temprature'].max()+1])))
fig.add_trace(go.Scatter(x=df1['Date'], y=df1['Temprature']), )
fig.update_layout(title='Temprature Throught Timeline:',
                 xaxis_title='Time', yaxis_title='Temprature in Degrees')
fig.update_layout(xaxis=go.layout.XAxis(
    rangeselector=dict(
        buttons=list([dict(label="Whole View", step="all"),
                      dict(count=1,label="One Year View",step="year",stepmode="todate")                      
                     ])),
        rangeslider=dict(visible=True),type="date")
)
fig.show()

**Note:** On a closer look, by clicking on One Year View, we can see that the graph seems distorted because this is how the values really are. The temperature varies every year with months.

**But, since this is not how seasons work. We have four main seasons in India and this is how they are grouped:**

* Winter : December, January and February.
* Summer(Also called, “Pre Monsoon Season”) : March, April and May.
* Monsoon : June, July, August and September.
* Autumn(Also called “Post Monsoon Season) : October and November.

# <center>**`Warmest/Coldest Average`**</center>

In [8]:
fig = px.box(df1, 'Month', 'Temprature')
fig.update_layout(title='Warmest, Coldest and Median Monthly Tempratue.')
fig.show()

* January has the coldest Days in an Year.
* May has the hottest days in an Year.
* *July* is the month with least Standard Deviation which means, temperature in July vary least. We can expect any day in July to be a warm day.

In [9]:
from sklearn.cluster import KMeans
sse = []
target = df1['Temprature'].to_numpy().reshape(-1,1)
num_clusters = list(range(1, 10))

for k in num_clusters:
    km = KMeans(n_clusters=k)
    km.fit(target)
    sse.append(km.inertia_)

fig = go.Figure(data=[
    go.Scatter(x = num_clusters, y=sse, mode='lines'),
    go.Scatter(x = num_clusters, y=sse, mode='markers')
])

fig.update_layout(title="Evaluation on number of clusters:",
                 xaxis_title = "Number of Clusters:",
                 yaxis_title = "Sum of Squared Distance",
                 showlegend=False)
fig.show()

**A cluster size of 3 seems a good choice here**

In [10]:
km = KMeans(3)
km.fit(df1['Temprature'].to_numpy().reshape(-1,1))
df1.loc[:,'Temp Labels'] = km.labels_
fig = px.scatter(df1, 'Date', 'Temprature', color='Temp Labels')
fig.update_layout(title = "Temprature clusters.",
                 xaxis_title="Date", yaxis_title="Temprature")
fig.show()

* Despite having 4 seasons we can see 3 main clusters based on temperatures.
* Jan, Feb and Dec are the coldest months.
* Apr, May, Jun, Jul, Aug and Sep; all have hotter temperatures.
* Mar, Oct and Nov are the months that have temperatures neither too hot nor too cold.

In [11]:
fig = px.histogram(x=df1['Temprature'], nbins=200, histnorm='density')
fig.update_layout(title='Frequency chart of temprature readings:',
                 xaxis_title='Temprature', yaxis_title='Count')

**There is a cluster from 26.2-27.5 and mean temperature for most months during history has been between 26.8-26.9**

# <center>**`Yearly average temperature`**</center>

In [12]:
df['Yearly Mean'] = df.iloc[:,1:].mean(axis=1) ## Axis 1 for row wise and axis 0 for columns.
fig = go.Figure(data=[
    go.Scatter(name='Yearly Tempratures' , x=df['YEAR'], y=df['Yearly Mean'], mode='lines'),
    go.Scatter(name='Yearly Tempratures' , x=df['YEAR'], y=df['Yearly Mean'], mode='markers')
])
fig.update_layout(title='Yearly Mean Temprature :',
                 xaxis_title='Time', yaxis_title='Temprature in Degrees')
fig.show()

* The yearly mean temperature was not increasing till 1980. It was only after 1979 that we can see the gradual increase in yearly mean temperature.
* After 2015, yearly temperature has increased drastically.
* We are seeing a monthly like up-down pattern in yearly temperatures as well.
* This is not understandable. Because with months, we have a phenomena of seasons and the earth the revolving around sun in a elliptic path. But this pattern is not expected in yearly temperature.

# <center>**`Monthly temperatures through history`**</center>

In [13]:
fig = px.line(df1, 'Year', 'Temprature', facet_col='Month', facet_col_wrap=4)
fig.update_layout(title='Monthly temprature throught history:')
fig.show()

**We can see clear positive trend lines. Let’s see if we could find any trend in seasonal mean temperatures.**

# <center>**`Seasonal Weather Analysis`**</center>

In [14]:
df['Winter'] = df[['DEC', 'JAN', 'FEB']].mean(axis=1)
df['Summer'] = df[['MAR', 'APR', 'MAY']].mean(axis=1)
df['Monsoon'] = df[['JUN', 'JUL', 'AUG', 'SEP']].mean(axis=1)
df['Autumn'] = df[['OCT', 'NOV']].mean(axis=1)
seasonal_df = df[['YEAR', 'Winter', 'Summer', 'Monsoon', 'Autumn']]
seasonal_df = pd.melt(seasonal_df, id_vars='YEAR', value_vars=seasonal_df.columns[1:])
seasonal_df.columns=['Year', 'Season', 'Temprature']

fig = px.scatter(seasonal_df, 'Year', 'Temprature', facet_col='Season', facet_col_wrap=2, trendline='ols')
fig.update_layout(title='Seasonal mean tempratures throught years:')
fig.show()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



**We can again see a positive trend line between temperature and time. The trend line does not have a very high positive correlation with years but still it is not negligible.**

In [15]:
px.scatter(df1, 'Month', 'Temprature', size='Temprature', animation_frame='Year')

**On first look, we can see some fluctuations but that doesn’t give much of insights for us. However, if we again see by arranging bar below to early years and late years we can notice the change.**

# <center>**`Weather Forecasting with Machine Learning`**</center>


```
Let’s try to forecast monthly mean temperature for year 2023.
```



In [16]:
# We am using decision tree regressor for prediction as the data does not actually have a linear trend.
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score 

df2 = df1[['Year', 'Month', 'Temprature']].copy()
df2 = pd.get_dummies(df2)
y = df2[['Temprature']]
x = df2.drop(columns='Temprature')

dtr = DecisionTreeRegressor()
train_x, test_x, train_y, test_y = train_test_split(x,y,test_size=0.3)
dtr.fit(train_x, train_y)
pred = dtr.predict(test_x)
r2_score(test_y, pred)

next_Year = df1[df1['Year']==2022][['Year', 'Month']]
next_Year.Year.replace(2022,2023, inplace=True)
next_Year= pd.get_dummies(next_Year)
temp_2023 = dtr.predict(next_Year)

temp_2023 = {'Month':df1['Month'].unique(), 'Temprature':temp_2023}
temp_2023=pd.DataFrame(temp_2023)
temp_2023['Year'] = 2023
temp_2023

,Month,Temprature,Year
0,JAN,22.106071,2023
1,FEB,24.702262,2023
2,MAR,25.145952,2023
3,APR,29.742619,2023
4,MAY,31.075833,2023
5,JUN,30.105119,2023
6,JUL,28.807143,2023
7,AUG,28.859762,2023
8,SEP,28.715238,2023
9,OCT,28.450000,2023


In [17]:
forecasted_temp = pd.concat([df1,temp_2023], sort=False).groupby(by='Year')['Temprature'].mean().reset_index()
fig = go.Figure(data=[
    go.Scatter(name='Yearly Mean Temprature', x=forecasted_temp['Year'], y=forecasted_temp['Temprature'], mode='lines'),
    go.Scatter(name='Yearly Mean Temprature', x=forecasted_temp ['Year'], y=forecasted_temp['Temprature'], mode='markers')
])
fig.update_layout(title='Forecasted Temprature:',
                 xaxis_title='Time', yaxis_title='Temprature in Degrees')
fig.show()

# <center>**`Project 2`**</center>

# <center>**`Live Weather report with the help of API`**</center>

**Receive current weather information of any place using openweather api**

> In this Python project, we will show a python app that will collect weather information such as current temperature, pressure, humidity, wind speed, weather description and many others, of any place on the earth, using OpenWeatherMap API.

We can find these weather information in real time by just typing the city name. We will be able to get the data about only those places which are listed by OpenWeatherMap website.



# Project requirements
* Python version 3.6 and above (although lower version should also work)
* Python requests module should be installed
* Free account on OpenWeatherMap and api access token

In [18]:
import requests
from datetime import datetime

api_key = '6f89b1f3c8543ded006c403c675457b9'
location = input("Enter the city name: ")

complete_api_link = "https://api.openweathermap.org/data/2.5/weather?q="+location+"&appid="+api_key
api_link = requests.get(complete_api_link)
api_data = api_link.json()

#create variables to store and display data
temp_city = ((api_data['main']['temp']) - 273.15)
weather_desc = api_data['weather'][0]['description']
hmdt = api_data['main']['humidity']
wind_spd = api_data['wind']['speed']
date_time = datetime.now().strftime("%d %b %Y | %I:%M:%S %p")

print ("Weather Stats for - {}  || {}".format(location.upper(), date_time))

print ("Current temperature is: {:.2f} deg C".format(temp_city))
print ("Current weather desc  :",weather_desc)
print ("Current Humidity      :",hmdt, '%')
print ("Current wind speed    :",wind_spd ,'kmph')



# making a list so that i can print the info to a text
txtlist = [temp_city,weather_desc,hmdt,wind_spd,date_time]

#using open() buit-in function to write to a text file
with open("textfile.txt" , mode= 'w' ,encoding= 'utf-8') as f :     

#encoding = utf-8 for linux and cp1252 for win
       
    f.write("Weather Stats for - {}  || {}".format(location.upper(), date_time))
    f.write("Current temperature is: {:.2f} deg C\n".format(txtlist[0]))
    
    f.write("{},{} \n".format("Current weather desc  :" ,txtlist[1]))
    f.write("{},{},{} \n".format("Current Humidity      :",txtlist[2],"%"))
    f.write("{},{},{} \n".format("Current wind speed    :",txtlist[3],"kmph"))

Enter the city name: chandigarh
Weather Stats for - CHANDIGARH  || 24 Jan 2022 | 09:35:27 AM
Current temperature is: 15.77 deg C
Current weather desc  : broken clouds
Current Humidity      : 64 %
Current wind speed    : 2.8 kmph
